In [19]:
import pandas as pd;
import numpy as np;
from transformers import BertTokenizer;

In [20]:
df=pd.read_csv("./review_dataset/train.tsv",delimiter="\t")
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [21]:
df.shape

(156060, 4)

In [22]:
df.drop_duplicates(subset="SentenceId",keep="first",inplace=True)

In [23]:
df["Sentiment"].value_counts()

3    2321
1    2200
2    1655
4    1281
0    1072
Name: Sentiment, dtype: int64

In [24]:
seq_len=512
num_samples=len(df)

In [25]:
tokenizer=BertTokenizer.from_pretrained('bert-base-cased')

In [26]:
tokens=tokenizer(df["Phrase"].tolist(),max_length=seq_len,padding="max_length",truncation=True,return_tensors="np",
                 add_special_tokens=True)

In [27]:
tokens.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [28]:
tokens["input_ids"]

array([[ 101,  138, 1326, ...,    0,    0,    0],
       [ 101, 1188, 3589, ...,    0,    0,    0],
       [ 101, 2431, 3899, ...,    0,    0,    0],
       ...,
       [ 101,  118,  149, ...,    0,    0,    0],
       [ 101, 1109, 2523, ...,    0,    0,    0],
       [ 101, 1109, 1273, ...,    0,    0,    0]])

In [29]:
tokens["attention_mask"]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [30]:
with open("movie-xids.npy","wb") as f:
    np.save(f,tokens["input_ids"])

In [31]:
with open("movie-xmask.npy","wb") as f:
    np.save(f,tokens["attention_mask"])

In [32]:
arr=df["Sentiment"].values

In [33]:
arr

array([1, 4, 1, ..., 1, 1, 2], dtype=int64)

In [34]:
from sklearn.preprocessing import OneHotEncoder;
encoder=OneHotEncoder(sparse=False)
labels=encoder.fit_transform(df["Sentiment"].to_numpy().reshape(-1,1))

In [35]:
labels

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [36]:
with open("movie_labels.npy","wb") as f:
    np.save(f,labels)